In [1]:
pip install firecrawl-py python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from firecrawl import FirecrawlApp
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = "fc-54426a76940c4642b6321da1902131fe"

# Initialize Firecrawl application
# app = FirecrawlApp(api_key=api_key)

# # Define the target URL
# url = "https://www.cbinsights.com/research/startup-failure-post-mortem/"

# # Scrape the URL
# response = app.scrape_url(url)

# # Extract and print the main content in Markdown format
# markdown_content = response.get('markdown')
# print(markdown_content)

In [ ]:
# Save the Markdown content to a file,
# create the output directory if it doesn't exist
# output_dir = "output"
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)
# output_file = "output/output.md"
# with open(output_file, "w") as file:
#     file.write(markdown_content)
# print(f"Markdown content saved to {output_file}")

Markdown content saved to output/output.md


In [19]:
file_path = "./output/output.md"

try:
    with open(file_path, "r") as file_variable:
        content = file_variable.read()
        print(content)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")



### **Olive**

[Olive](https://www.cbinsights.com/company/crosschx) offered healthcare AI solutions that automated key tasks like prior authorization approvals and clinical analysis for surgeries. It consistently drew fresh capital over its lifetime, raising a new round nearly every year following its founding in 2012. However, the combination of challenging economic conditions, evolving customer expectations, and management missteps led the company to sell off its core assets and shut down.

- **Select VC investors:** Silicon Valley Bank, Google Ventures, Sequoia Capital, Tiger Global Management
- **Total equity funding:** $850M
- **Peak valuation:** $4B

[![CBI customers can click this banner to access instant insights on Olive's funding history](https://research-assets.cbinsights.com/2024/05/28183836/olive-banner-1024x77.png)](https://app.cbinsights.com/profiles/c/ven3M/fundings)

#### **Instant Insights: Olive exits market, sells off main business units in October 2023**

- Olive

In [20]:
# spllit the content by 30 h3 headings
def split_by_h3_headings(markdown_content, headings_per_chunk=30):
    """
    Splits markdown content into chunks where each chunk contains up to 
    the specified number of h3 headings.
    
    Args:
        markdown_content (str): The markdown content to split
        headings_per_chunk (int, optional): Number of h3 headings per chunk. Defaults to 30.
    
    Returns:
        list: A list of markdown chunks as strings
    """
    # Split the content by lines
    lines = markdown_content.split('\n')
    
    chunks = []
    current_chunk = []
    h3_count = 0
    
    for line in lines:
        # Check if the line is an h3 heading (starts with ###)
        is_h3 = line.lstrip().startswith('### ')
        
        # If we hit an h3 heading and already have the max h3s in the current chunk,
        # save the current chunk and start a new one
        if is_h3:
            h3_count += 1
            
            if h3_count > headings_per_chunk and h3_count % headings_per_chunk == 1:
                # Save current chunk when we hit the 31st, 61st, etc. h3 heading
                chunks.append('\n'.join(current_chunk))
                current_chunk = []
        
        # Add the current line to the current chunk
        current_chunk.append(line)
    
    # Add the last chunk if it has content
    if current_chunk:
        chunks.append('\n'.join(current_chunk))
    
    return chunks



In [21]:
split_content=split_by_h3_headings(content, headings_per_chunk=30)

In [22]:
split_content[0]

"\n\n### **Olive**\n\n[Olive](https://www.cbinsights.com/company/crosschx) offered healthcare AI solutions that automated key tasks like prior authorization approvals and clinical analysis for surgeries. It consistently drew fresh capital over its lifetime, raising a new round nearly every year following its founding in 2012. However, the combination of challenging economic conditions, evolving customer expectations, and management missteps led the company to sell off its core assets and shut down.\n\n- **Select VC investors:** Silicon Valley Bank, Google Ventures, Sequoia Capital, Tiger Global Management\n- **Total equity funding:** $850M\n- **Peak valuation:** $4B\n\n[![CBI customers can click this banner to access instant insights on Olive's funding history](https://research-assets.cbinsights.com/2024/05/28183836/olive-banner-1024x77.png)](https://app.cbinsights.com/profiles/c/ven3M/fundings)\n\n#### **Instant Insights: Olive exits market, sells off main business units in October 20

In [ ]:
template_string = """As a professional summarizer, create a concise and comprehensive summary of the information for each failed startups in this webpadge, while adhering to these guidelines:\

**As a professional summarizer, create a concise and comprehensive summary of the information for each failed startups in this webpadge, while adhering to these guidelines:

**All summaries must be about healthcare startups, exclude startups that is not in the healthcare filed**scan through the content and generate a summary for each startup failure, focusing on the key points and essential information.

1. **Detailed and Clear**  
   Craft a summary that is thorough, in-depth, and complex while maintaining clarity and conciseness.

2. **Focus on Main Ideas**  
   Incorporate key points and essential information, ensure to cover all information, emphasizing on critical aspects。

3. **Strictly Text-Based**  
   Rely exclusively on the provided text without adding external information.

4. **Paragraph Formatting**  
   Format the summary of each key point into a clear and readable paragraph.

Webpadge: ```{Webpadge}```

{format_instructions}
"""

In [51]:
from langchain.output_parsers import PydanticOutputParser 
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser 
from pydantic import BaseModel, Field, field_validator
from typing import List
from langchain_openai import ChatOpenAI


import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

llm = ChatOpenAI(model="gpt-4o",temperature=0.5)

In [52]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [53]:
class startup_data(BaseModel):
    startup_name: str = Field(description="The name of the startup")
    startup_description: str = Field(description="The description of the startup")
    startup_vc: str = Field(description="The name of the VC investors of the startup")
    startup_funding: str = Field(description="The funding amount of the startup")
    startup_peak_value: str = Field(description="The peak value of the startup")
    startup_failure_reason: str = Field(description="The reason for the startup's failure, make sure the reason is comperhensive and clear")
    # You can add custom validation logic easily with Pydantic.
pydantic_parser = PydanticOutputParser(pydantic_object=startup_data)

format_instructions = pydantic_parser.get_format_instructions()

In [54]:
prompt = ChatPromptTemplate.from_template(template=template_string)

messages = prompt.format_messages(Webpadge=split_content[0],
                                format_instructions=format_instructions)

In [55]:
print(messages[0].content)

As a professional summarizer, create a concise and comprehensive summary of the information for each failed startups in this webpadge, while adhering to these guidelines:
**All summaries must be about healthcare startups, exclude startups that is not in the healthcare filed**Scan through the webpadge and generate a summary for each startup cases, do not miss any startup cases.
**Summary Guidelines:**
1. **Detailed and Clear**  
   Craft a summary that is thorough, in-depth, and complex while maintaining clarity and conciseness.

2. **Focus on Main Ideas**  
   Incorporate key points and essential information, ensure to cover all information, emphasizing on critical aspects。

3. **Strictly Text-Based**  
   Rely exclusively on the provided text without adding external information.

4. **Paragraph Formatting**  
   Format the summary of each key point into a clear and readable paragraph.

Webpadge: ```

### **Olive**

[Olive](https://www.cbinsights.com/company/crosschx) offered healthcar

In [56]:
# output=llm(messages)

output=llm.invoke(messages)
print(output.content)

# print(json_output)

```json
[
    {
        "startup_name": "Olive",
        "startup_description": "Olive offered healthcare AI solutions that automated key tasks like prior authorization approvals and clinical analysis for surgeries.",
        "startup_vc": "Silicon Valley Bank, Google Ventures, Sequoia Capital, Tiger Global Management",
        "startup_funding": "$850M",
        "startup_peak_value": "$4B",
        "startup_failure_reason": "Olive faced challenging economic conditions, evolving customer expectations, and management missteps which led to the sale of its core business units and eventual shutdown."
    },
    {
        "startup_name": "Avail",
        "startup_description": "Avail developed technologies that enabled clinicians to communicate with parties outside the operating room, such as medical device sales reps and medical advisors.",
        "startup_vc": "D1 Capital Partners, Playground Global, Coatue, Baidu Ventures",
        "startup_funding": "$138M",
        "startup_peak_value

In [17]:
#use the json parser to parse the output


from langchain_core.output_parsers import JsonOutputParser
json_output = JsonOutputParser()
parsed_output = json_output.parse(output)
print(parsed_output)



NameError: name 'output' is not defined